In [ ]:
import warnings
warnings.filterwarnings("ignore")

import json
from os.path import join

from deepburtsev.core.pipelinemanager import PipelineManager
from deepburtsev.core.transformers import FasttextVectorizer, ResultsCollector
from deepburtsev.models.intent_classification.DCNN import DCNN
from deepburtsev.models.skmodels.linear_models import LinearRegression, LinearSVM, RandomForest
from deepburtsev.core.sktransformers import Tfidf
from deepburtsev.core.sktransformers import Count

### Объявляем путь к датасету и корневой репозиторий

In [ ]:
root = '/home/mks/projects/DeepBurtsev/'
file_path = join(root, 'data', 'english', 'new_group', 'dataset.json')
with open(file_path, 'r') as f:
    dataset = json.load(f)
    f.close()

### Скачивае ембеддинг по ссылке

In [ ]:
from deepburtsev.core.utils import download_untar
url = 'http://lnsigo.mipt.ru/export/embeddings/ft_native_300_ru_wiki_lenta_nltk_word_tokenize/ft_native_300_ru_wiki_lenta_nltk_word_tokenize.bin'
download_path = join(root, 'downloads', 'embeddings')
download_untar(url, download_path)

### Задём недефолтные значения операциям, если хотим.

In [ ]:
fasttext = FasttextVectorizer(request_names=['train', 'valid', 'test'],
                              new_names=['train', 'valid', 'test'],
                              dimension=300,
                              model_path='./downloads/embeddings/ft_native_300_ru_wiki_lenta_nltk_word_tokenize.bin')

tfidf = Tfidf(request_names=['train', 'valid', 'test'], new_names=['train', 'valid', 'test'])
count = Count(request_names=['train', 'valid', 'test'], new_names=['train', 'valid', 'test'])

### Задаём структуру эксперимента при помощи списка операций

In [ ]:
# create structure for pipeline manager
neural_struct = [fasttext, (DCNN, {'search': True, 'batch_size': 32, 'epochs': [3, 5, 8, 10, 12, 14, 16, 18, 20]}),
                 ResultsCollector]

linear_struct = [[tfidf, count],
                 [LinearRegression, LinearSVM, RandomForest],
                 ResultsCollector]

In [ ]:
# Запуск перебора всех пайплайнов, параметр hyper_search отвечает за включение и выключение подбора гиперпараметров
# У нейронной модели, по дефолту стоит 20 эпох. Если включить подбор гиперпараметров,
# то считаться всё будет очень долго

neural_man = PipelineManager(dataset, neural_struct, 'skill_manager', target_metric='f1_macro')
neural_man.run()

linear_man = PipelineManager(dataset, linear_struct, 'skill_manager', target_metric='f1_macro')
linear_man.run()

### Вывод результатов